In [1]:
from langchain.memory import ConversationBufferMemory

In [2]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [3]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [4]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

続いて、Chainに対してhistoryを引数で与えた場合に同様の返事ができるかを実験する。次の3通りを試す。

1. memoryを与えない場合
2. memoryを与えず、順に質問する場合
3. memoryをインスタンス化の引数で与えた場合

In [5]:
llm1 = OpenAI(temperature=0)
conversation1 = ConversationChain(
    llm=llm1, 
    verbose=True, 
)
conversation1.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's nice to meet you. I'm an AI that specializes in providing detailed information about my context. What would you like to know?"

In [12]:
llm2 = OpenAI(temperature=0)
conversation2 = ConversationChain(
    llm=llm2, 
    verbose=True, 
)
conversation2.predict(input="Hi there!")
conversation2.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

In [13]:
llm3 = OpenAI(temperature=0)
memory3 = ConversationBufferMemory()
memory3.chat_memory.add_user_message("Hi there!")
conversation3 = ConversationChain(
    llm=llm3,
    verbose=True, 
    memory=memory3
)
conversation3.predict(input="I'm doing well! Just having a conversation with an AI.")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" Hi there! It's nice to meet you. I'm an AI, and I'm doing great! I'm excited to have a conversation with you. What would you like to talk about?"

結論として、Chainに対してメモリーを渡さない場合、少なくとも ConversationChain では自動でメモリを使って返事するようにはなっていないことがわかる。

## memory_key の機能を探る

In [16]:
# [LangChain のメモリについて | Hakky Handbook](https://book.st-hakky.com/docs/memory-of-langchain/)

# `ConversationBufferMemory` のインポート
from langchain.memory import ConversationBufferMemory

# `ConversationBufferMemory` の初期化
memory = ConversationBufferMemory(human_prefix="User", ai_prefix="Bot", memory_key="history")

# ユーザーメッセージと AI メッセージの追加
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

# 保存されたメッセージを変数に読み込む
buffer = memory.load_memory_variables({})
print(buffer)
print(buffer["history"])

{'history': 'User: hi!\nBot: whats up?'}
User: hi!
Bot: whats up?


In [17]:
# これ、memory_key どうして必要なんだ？例えば存在しない memory_keyを指定したら？
print(buffer["nothing"])

KeyError: 'nothing'

In [18]:
# もしかして、メッセージ追加時に memory_key を指定することができる？
memory.chat_memory.add_user_message("こんにちは!", memory_key="history2")

TypeError: ChatMessageHistory.add_user_message() got an unexpected keyword argument 'memory_key'

In [19]:
# では、Memoryオブジェクトが別でもBufferは同じ空間に保存されている、という仮説はどうだろうか。だとしたらConversationBufferMemory()のインスタンス時に引数を渡すのも納得できる。
memory1 = ConversationBufferMemory(human_prefix="User", ai_prefix="Bot", memory_key="history1")
memory1.chat_memory.add_user_message("hi!")

memory2 = ConversationBufferMemory(human_prefix="User", ai_prefix="Bot", memory_key="history2")
memory2.chat_memory.add_user_message("こんにちは!")

buffer = memory1.load_memory_variables({})
print(buffer)

{'history1': 'User: hi!'}


メモリクラスを自前で実装する方法も見たが、正直言って memory_key の存在異議はよく分からない...

[LangChain の HOW-TO EXAMPLES (6) - メモリ｜npaka｜note](https://note.com/npaka/n/n155e66a263a2)

SlackBotで利用する上では、スレッドごとに複数のエージェントを用いるのがよさそうだ。

## Agentのrun()実行時に、memory_keyを渡してもダメなのか実験

In [23]:
from langchain.agents import Tool, initialize_agent
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()
agent_chain = initialize_agent(
    [Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Useful for finding information about a specific topic. You cannot use this tool to ask questions, only to find information about a specific topic.",
    )],
    OpenAI(temperature=0),
    agent="chat-conversational-react-description",
    verbose=True,
    memory=ConversationBufferMemory(
        memory_key="chat_history", return_messages=True),
)

agent_chain.run(input="こんにちは。HUNTERxHUNTERのあらすじを教えて下さい。")
agent_chain.run(input="どんな登場人物がいますか？")   # 文脈を掴んで、HUNTERxHUNTERの登場人物を言うはず
# 文脈を見失うはず（冨樫義博と答えたら失敗）
agent_chain.run(input="作者は誰ですか？", memory_key="another_history")




> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "Wikipedia",
    "action_input": "Hunter x Hunter"
}
```
Observation: Page: Hunter × Hunter
Summary: Hunter × Hunter (stylized as HUNTER×HUNTER and pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 37 tankōbon volumes as of November 2022. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually

ValueError: Could not parse LLM output: 

RESPONSE
--------------------
```json
{
    "action": "Wikipedia",
    "action_input": "Yoshihiro Togashi"
}
``

エラーではあるが、冨樫義博先生の名前を答えようとしているので、memory_keyをrun()で与えても特に意味はなさそうであることが分かる。